In [18]:
import os
import sys
import json
import urllib.request
from datetime import date
import pandas as pd
from dateutil.relativedelta import relativedelta
from tqdm import tqdm_notebook

df1 = pd.read_csv('movie_04.csv')
start_date = date(2017,1,1)
end_date = date(2017,12,31)
len(df1)

269

In [19]:
m_name = df1['movieNm'][0]
m_code = df1['movieCd'][0]
n_code = df1['naver_movieCd'][0]
m_code

20165748

In [20]:
with open("/root/movie/movie_info/"+str(m_code)+".json") as data_file:    
    data = json.load(data_file)
openDt = data['movieInfoResult']['movieInfo']['openDt']
openDt


UnicodeDecodeError: 'ascii' codec can't decode byte 0xea in position 82: ordinal not in range(128)

In [7]:
date(int(str(openDt)[0:4]),int(str(openDt)[4:6]),int(str(openDt)[6:8]))
edt = date(int(str(openDt)[0:4]),int(str(openDt)[4:6]),int(str(openDt)[6:8])) + relativedelta(months=1)
sdt = date(int(str(openDt)[0:4]),int(str(openDt)[4:6]),int(str(openDt)[6:8]))
str(sdt)

'2017-11-22'

In [8]:
type(str(n_code))

str

In [9]:
for i in tqdm_notebook(range(len(df1['naver_movieCd']))):
    m_name = df1['movieNm'][i]
    m_code = df1['movieCd'][i]
    n_code = df1['naver_movieCd'][i]
    
    with open('movie_info/'+str(m_code)+'.json') as data_file:    
        data = json.load(data_file)
    openDt = data['movieInfoResult']['movieInfo']['openDt']
    
    idt = date(2016,1,1)
    sdt = date(int(str(openDt)[0:4]),int(str(openDt)[4:6]),int(str(openDt)[6:8]))
    edt = sdt + relativedelta(months=1)
    
    if(sdt > idt):
        client_id = "nA8Iwm4HJ6EM2W1dueU9"
        client_secret = "HaEPQCPbyQ"
        url = "https://openapi.naver.com/v1/datalab/search"

        # body='{\"startDate\":\"'+str(sdt)+'\",\"endDate\":\"2017-01-21\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\"영화\",\"keywords\":[\"'+'꾼'+'\"]}]}'
        body='{\"startDate\":\"'+str(sdt)+'\",\"endDate\":\"'+str(edt)+'\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\"영화\",\"keywords\":[\"'+m_name+'\"]}]}'

        print(body)
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()

            d = json.loads(response_body)
        #     print(response_body.decode('utf-8'))
#             with open('data2/datalab/'+str(n_code)+'.json', 'w', encoding="utf-8") as make_file:
#                 json.dump(d, make_file, ensure_ascii=False, indent="\t")
        else:
            print("Error Code:" + rescode)
            
        d['results'][0]['data']
        df = pd.DataFrame(d['results'][0]['data'])
        df.to_csv('data2/datalab2/'+str(n_code)[:-2]+'_datalab.csv',index=False)
    else:
        pass

{"startDate":"2017-11-22","endDate":"2017-12-22","timeUnit":"date","keywordGroups":[{"groupName":"영화","keywords":["꾼"]}]}
{"startDate":"2017-11-29","endDate":"2017-12-29","timeUnit":"date","keywordGroups":[{"groupName":"영화","keywords":["기억의 밤"]}]}
{"startDate":"2017-11-29","endDate":"2017-12-29","timeUnit":"date","keywordGroups":[{"groupName":"영화","keywords":["오리엔트 특급 살인"]}]}
{"startDate":"2017-11-29","endDate":"2017-12-29","timeUnit":"date","keywordGroups":[{"groupName":"영화","keywords":["반드시 잡는다"]}]}
{"startDate":"2017-11-15","endDate":"2017-12-15","timeUnit":"date","keywordGroups":[{"groupName":"영화","keywords":["저스티스 리그"]}]}
{"startDate":"2017-11-09","endDate":"2017-12-09","timeUnit":"date","keywordGroups":[{"groupName":"영화","keywords":["러빙 빈센트"]}]}
{"startDate":"2017-10-25","endDate":"2017-11-25","timeUnit":"date","keywordGroups":[{"groupName":"영화","keywords":["토르: 라그나로크"]}]}
{"startDate":"2017-11-08","endDate":"2017-12-08","timeUnit":"date","keywordGroups":[{"groupName":"영화","keywo

In [10]:
len(df1)

269

In [76]:
from tqdm import tqdm_notebook
df1 = pd.read_csv('movie_04.csv')

for i in tqdm_notebook(range(len(df1))): #range(len(df1)):
    ncd = str(df1['naver_movieCd'][i])[:-2]
#     print(ncd)
    try:
        df2 = pd.read_csv('/root/movie/data2/datalab2/'+ncd+'_datalab.csv')
        df3 = pd.read_csv('/root/movie/data2/after_rating/'+ncd+'_rating.csv')
    
        for i in range(len(df3)):
            df3['0'][i] = str(df3['0'][i])[0:4]+"-"+str(df3['0'][i])[4:6]+"-"+str(df3['0'][i])[6:8]
        df3.rename(columns={"0":"period"}, inplace = True)
        df4 = pd.merge(df2,df3,how='left',on='period')
        for i in range(len(df4)):
            df4['1'][i] = df4['1'][i] * 100
            df4['2'][i] = df4['2'][i] * 10
        df4.to_csv('/root/movie/data2/merge/'+ncd+'_merge.csv',index=False)
    except FileNotFoundError:
        pass

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
